In [ ]:

from pydub import AudioSegment
import speech_recognition as sr
import os

def split_audio(file_path, chunk_length_ms=5000):
    audio = AudioSegment.from_wav(file_path)
    chunks = []
    for i in range(0, len(audio), chunk_length_ms):
        chunk = audio[i:i + chunk_length_ms]
        chunk_name = f"chunk_{i // chunk_length_ms}.wav"
        chunk.export(chunk_name, format="wav")
        chunks.append((chunk_name, i // 1000))
    return chunks

def transcribe_chunks(chunks):
    r = sr.Recognizer()
    results = []
    for file, start_time in chunks:
        with sr.AudioFile(file) as source:
            audio = r.record(source)
        try:
            text = r.recognize_google(audio)
            results.append((start_time, text))
        except sr.UnknownValueError:
            results.append((start_time, "[Unclear]"))
        os.remove(file)
    return results

def search_keyword(transcripts, keyword):
    return [(t, txt) for t, txt in transcripts if keyword.lower() in txt.lower()]

file_path = "audiofile.wav"
chunks = split_audio(file_path)
transcripts = transcribe_chunks(chunks)
keyword = input("Enter keyword to search: ")
results = search_keyword(transcripts, keyword)
for time, text in results:
    print(f"{time}s: {text}")
